<a href="https://colab.research.google.com/github/chrbertsch/fmi3-features/blob/main/adjoint_derivatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adjoint Derivatives

This notebook demonstrates how to efficiently retrieve [adjoint derivative](https://fmi-standard.org/docs/3.0/#partial-derivatives) using the `fmi3GetAdjointDerivative()` function introduced in FMI 3.0.
We'll use the [Van der Pol oscillator](https://en.wikipedia.org/wiki/Van_der_Pol_oscillator) model from the [Reference FMUs](https://github.com/modelica/Reference-FMUs) that implements the following equation:

```
der(x0) = x1
der(x1) = mu * ((1 - x0 * x0) * x1) - x0
```

In [20]:
!uv pip install fmpy[complete] --system
!git clone https://github.com/chrbertsch/fmi3-features

Using Python 3.12.11 environment at: /usr
Audited 1 package in 297ms
fatal: destination path 'fmi3-features' already exists and is not an empty directory.


In [21]:
from fmpy import *
from shutil import rmtree
import plotly.graph_objects as go
import numpy as np


fmu_filename = 'fmi3-features/resources/VanDerPol.fmu'

# change the start values, so we can see how the states converge
result = simulate_fmu(fmu_filename, start_values={'x0': 0, 'x1': 4})

fig = go.Figure()

# plot the trajectory of the two states
fig.add_trace(go.Scatter(x=result['x0'], y=result['x1']))

axes_attrs = dict(showgrid=True, gridwidth=1, ticklen=0, gridcolor='LightGrey', linecolor='black', showline=True, zerolinewidth=1, zerolinecolor='LightGrey')
fig.update_xaxes(**axes_attrs)
fig.update_yaxes(**axes_attrs)
fig.update_layout(margin=dict(t=30, b=0, r=30, l=0), plot_bgcolor='rgba(0,0,0,0)', xaxis_title="x0", yaxis_title="x1")

fig.show()

In [22]:
# extract the FMU
unzipdir = extract(fmu_filename)

# load the model description
model_description = read_model_description(unzipdir)

# build a dictionary of (variable_name -> value_reference)
vr = dict((v.name, v.valueReference) for v in model_description.modelVariables)

# instantiate the FMU
fmu_instance = instantiate_fmu(unzipdir=unzipdir,
                               model_description=model_description,
                               fmi_call_logger=print,     # print FMI calls to the console
                               fmi_type='ModelExchange')  # workaround for https://github.com/modelica/Reference-FMUs/issues/245

# initialize the FMU instance
fmu_instance.enterInitializationMode()
fmu_instance.exitInitializationMode()

fmi3InstantiateModelExchange(instanceName="VanDerPol", instantiationToken="{8c4e810f-3da3-4a00-8276-176fa3c9f000}", resourcePath="/tmp/tmpkb0tqypu/resources/", visible=False, loggingOn=False, instanceEnvironment=0x0, logMessage=<CFunctionType object at 0x7d9f5acea8d0>) -> 0x2b8d30d0
fmi3EnterInitializationMode(instance=0x2b8d30d0, toleranceDefined=False, tolerance=0.0, startTime=0.0, stopTimeDefined=False, stopTime=0.0) -> OK
fmi3ExitInitializationMode(instance=0x2b8d30d0) -> OK


In [ ]:
knowns   = [vr['x0'],      vr['x1']     ]  # value references of the continuous states
unknowns = [vr['der(x0)'], vr['der(x1)']]  # value references of the continuous state derivatives
seed     = [1, 1]                          # dummy seed values (typically provided by algorithm)

# create a 2x2 matrix of zeros
J = np.zeros((len(knowns), len(knowns)))

# construct the Jacobian matrix column wise (FMI 2.0 method)
for i, known in enumerate(knowns):
    J[:, i] = fmu_instance.getDirectionalDerivative(unknowns, [known], [1])

# calculate v^T * J
sensitivity = seed @ J

sensitivity

fmi3GetDirectionalDerivative(instance=0x1628e2b02f0, unknowns=[2, 4], nUnknowns=2, knowns=[1], nKnowns=1, seed=[1.0], nSeed=1, sensitivity=[0.0, -1.0], nSensitivity=2) -> OK
fmi3GetDirectionalDerivative(instance=0x1628e2b02f0, unknowns=[2, 4], nUnknowns=2, knowns=[3], nKnowns=1, seed=[1.0], nSeed=1, sensitivity=[1.0, -3.0], nSensitivity=2) -> OK


array([-1., -2.])

In [23]:
# retrive v^T * J with a single call
sensitivity = fmu_instance.getAdjointDerivative(unknowns, knowns, seed)

sensitivity

fmi3GetAdjointDerivative(instance=0x2b8d30d0, unknowns=[2, 4], nUnknowns=2, knowns=[1, 3], nKnowns=2, seed=[1.0, 1.0], nSeed=2, sensitivity=[-1.0, -2.0], nSensitivity=2) -> OK


[-1.0, -2.0]

In [24]:
# clean up
fmu_instance.freeInstance()
rmtree(unzipdir)

fmi3FreeInstance(instance=0x2b8d30d0)
